In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
import matplotlib.pyplot as plt

In [2]:
# read data
df = pd.read_csv('analytics_20191001_20201001.csv', skiprows=6).dropna()
# convert date to datetime object
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
# character modifications for string columns
df['Users'] = df['Users'].apply(lambda x: x.replace(',', ''))
df['New Users'] = df['New Users'].apply(lambda x: x.replace(',', ''))
df['Sessions'] = df['Sessions'].apply(lambda x: x.replace(',', ''))
df['Bounce Rate'] = df['Bounce Rate'].apply(lambda x: x.replace('%', ''))
df['Ecommerce Conversion Rate'] = (df['Ecommerce Conversion Rate'].apply(lambda x: x.replace('%', '')))
df['Revenue'] = (df['Revenue'].apply(lambda x: x.replace('$', '')))
df['Revenue'] = (df['Revenue'].apply(lambda x: x.replace(',', '')))
# type and value modifications
df['Users'] = df['Users'].astype(int)
df['New Users'] = df['New Users'].astype(int)
df['Sessions'] = df['Sessions'].astype(int)
df['Bounce Rate'] = (df['Bounce Rate'].astype(float)) / 100
df['Ecommerce Conversion Rate'] = (df['Ecommerce Conversion Rate'].astype(float)) / 100
df['Revenue'] = df['Revenue'].astype(float)

# helper function for string --> time conversion
def to_seconds(time_str):
    # convert time string to datetime object
    date_time = datetime.datetime.strptime(time_str, "%H:%M:%S")
    # datetime object defaults to 1900-01-01, so we can use that to get the length of time in the time string
    a_timedelta = date_time - datetime.datetime(1900, 1, 1)
    # conver to seconds and return
    seconds = a_timedelta.total_seconds()
    return seconds

# convert session duration to seconds
df['Avg. Session Duration'] = df['Avg. Session Duration'].apply(lambda x: x.replace('<00:00:01', '00:00:01'))
df['Avg. Session Duration'] = df['Avg. Session Duration'].apply(to_seconds)

df.head()

,Default Channel Grouping,Date,Date Range,Segment,Users,New Users,Sessions,Bounce Rate,Pages / Session,Avg. Session Duration,Ecommerce Conversion Rate,Transactions,Revenue
0,Organic Search,2019-12-02,"Oct 1, 2019 - Jan 1, 2020",All Users,1828,1463,1973,0.5479,3.47,143.0,0.0015,3.0,73.8
1,Organic Search,2019-12-02,"Oct 1, 2019 - Jan 1, 2020",Made a Purchase,3,3,3,0.0000,14.33,343.0,1.0000,3.0,73.8
2,Organic Search,2019-12-02,"Oct 1, 2019 - Jan 1, 2020",Tablet and Desktop Traffic,1345,1052,1457,0.5484,3.59,158.0,0.0014,2.0,32.0
3,Organic Search,2019-12-02,"Oct 1, 2019 - Jan 1, 2020",Add to Cart,47,27,48,0.0000,15.40,763.0,0.0000,0.0,0.0
4,Organic Search,2019-11-29,"Oct 1, 2019 - Jan 1, 2020",All Users,1814,1527,1889,0.5648,3.07,106.0,0.0016,3.0,139.0


# Data Wrangling

Aggregate by date and channel, summing users:

In [3]:
# add number of users from each channel
df_channel_agg = df.groupby(['Date', 'Default Channel Grouping']).agg({'Users': np.sum})
# transpose user counts into columns
df_channel_agg = df_channel_agg.groupby(['Date', 'Default Channel Grouping'])['Users'].first().unstack(fill_value='').rename_axis('Date')
# make date index a column (easier for merging later)
df_channel_agg = df_channel_agg.reset_index()
# no need to name the columns
df_channel_agg.columns.name = ''
# rename columns
df_channel_agg = df_channel_agg.rename(columns={
    'Date': 'date',
    '(Other)': 'user_other',
    'Affiliates': 'user_affiliate',
    'Direct': 'user_direct',
    'Display': 'user_display',
    'Organic Search': 'user_organic',
    'Paid Search': 'user_paid',
    'Referral': 'user_referral',
    'Social': 'user_social'
})
df_channel_agg.head()

,date,user_other,user_affiliate,user_direct,user_display,user_organic,user_paid,user_referral,user_social
0,2019-10-01,120,149,670,33,2196,245,855,109
1,2019-10-02,110,158,672,39,2152,214,824,103
2,2019-10-03,81,132,729,43,2132,269,707,639
3,2019-10-04,89,104,547,34,1737,205,692,139
4,2019-10-05,60,64,315,16,1201,125,197,95


Aggregate by date and segment, sum users:

In [4]:
# add number of users from each channel
df_seg_agg = df.groupby(['Date', 'Segment']).agg({'Users': np.sum})
# transpose user counts into columns
df_seg_agg = df_seg_agg.groupby(['Date', 'Segment'])['Users'].first().unstack(fill_value='').rename_axis('Date')
# make date index a column (easier for merging later)
df_seg_agg = df_seg_agg.reset_index()
# no need to name the columns
df_seg_agg.columns.name = ''
# no need for the all users column
df_seg_agg = df_seg_agg.drop(columns=['All Users'])
# rename columns
df_seg_agg = df_seg_agg.rename(columns={
    'Date': 'date',
    'Add to Cart': 'user_addtocart',
    'Made a Purchase': 'user_purchase',
    'Tablet and Desktop Traffic': 'user_tabletdesktop'
})
df_seg_agg.head()

,date,user_addtocart,user_purchase,user_tabletdesktop
0,2019-10-01,105,5,1868
1,2019-10-02,107,3,1800
2,2019-10-03,95,6,1899
3,2019-10-04,97,3,1438
4,2019-10-05,58,4,741


Drop unneeded columns:

In [5]:
# remove columns that we don't need from df
df_for_agg = df.drop(columns=['Default Channel Grouping', 'Date Range', 'Segment', 'Sessions',
                      'Ecommerce Conversion Rate', 'Transactions', 'Revenue'])
df_for_agg.head()

,Date,Users,New Users,Bounce Rate,Pages / Session,Avg. Session Duration
0,2019-12-02,1828,1463,0.5479,3.47,143.0
1,2019-12-02,3,3,0.0000,14.33,343.0
2,2019-12-02,1345,1052,0.5484,3.59,158.0
3,2019-12-02,47,27,0.0000,15.40,763.0
4,2019-11-29,1814,1527,0.5648,3.07,106.0


Group by date and apply aggregation functions:

In [6]:
# aggregation functions
df_agg = df_for_agg.groupby('Date').agg({
    'Users': np.sum,
    'New Users': np.sum,
    'Bounce Rate': np.mean,
    'Avg. Session Duration': np.mean,
    'Pages / Session': np.mean
})
# reset index
df_agg = df_agg.reset_index()
# rename columns
df_agg = df_agg.rename(columns={
    'Date': 'date',
    'Users': 'user',
    'New Users': 'user_new',
    'Bounce Rate': 'session _bounce',\
    'Avg. Session Duration': 'session_duration',
    'Pages / Session': 'session_depth'
})
df_agg.head()

,date,user,user_new,session _bounce,session_duration,session_depth
0,2019-10-01,4377,3210,0.270491,192.90625,4.826875
1,2019-10-02,4272,3173,0.245378,354.53125,7.964375
2,2019-10-03,4732,3722,0.249891,279.56250,5.829688
3,2019-10-04,3547,2619,0.236200,259.28125,5.872188
4,2019-10-05,2073,1595,0.222594,246.18750,4.771875


Merging all 3 dataframes:

In [7]:
df_stage = pd.merge(df_agg, df_channel_agg, on='date')
df_final = pd.merge(df_stage, df_seg_agg, on='date')
df_final

,date,user,user_new,session _bounce,session_duration,session_depth,user_other,user_affiliate,user_direct,user_display,user_organic,user_paid,user_referral,user_social,user_addtocart,user_purchase,user_tabletdesktop
0,2019-10-01,4377,3210,0.270491,192.90625,4.826875,120,149,670,33,2196,245,855,109,105,5,1868
1,2019-10-02,4272,3173,0.245378,354.53125,7.964375,110,158,672,39,2152,214,824,103,107,3,1800
2,2019-10-03,4732,3722,0.249891,279.56250,5.829688,81,132,729,43,2132,269,707,639,95,6,1899
3,2019-10-04,3547,2619,0.236200,259.28125,5.872188,89,104,547,34,1737,205,692,139,97,3,1438
4,2019-10-05,2073,1595,0.222594,246.18750,4.771875,60,64,315,16,1201,125,197,95,58,4,741
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,2019-12-28,1589,1275,0.280762,256.06250,7.096875,43,34,256,32,972,10,174,68,49,4,527
89,2019-12-29,1789,1419,0.268697,208.25000,4.815313,54,51,276,45,1105,2,176,80,32,4,662
90,2019-12-30,2654,2070,0.248406,302.40625,5.741875,73,49,370,32,1568,4,481,77,44,4,1044
91,2019-12-31,1474,1150,0.236409,333.78125,5.688125,33,44,265,13,875,3,154,87,23,0,521


# Analysis

(a)

In [8]:
# before running the regression, we'll add some time effect variables
# get day of week
df_final['day'] = df_final['date'].dt.day_name()
# create weekend dummy variable
df_final['weekend'] = ((df_final['day'] == 'Saturday') | (df_final['day'] == 'Sunday')).astype(int)
# get month of year
df_final['month'] = df_final['date'].apply(lambda x: x.month)
# create month dummy variable
df_final = pd.get_dummies(df_final, columns=['month'], drop_first=True, prefix='month')

# add holiday dummy
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
cal = calendar()
holidays = cal.holidays(start=df_final['date'].min(), end=df_final['date'].max())
# for some reason, pandas says 10-14 is a holiday and 10-31 is not; let's fix this
holidays = list(holidays) # convert to list for mutability
holidays[0] = pd.to_datetime('20191031', format='%Y%m%d')
df_final['holiday'] = df_final['date'].isin(holidays).astype(int)

df_final.head()

,date,user,user_new,session _bounce,session_duration,session_depth,user_other,user_affiliate,user_direct,user_display,...,user_social,user_addtocart,user_purchase,user_tabletdesktop,day,weekend,month_10,month_11,month_12,holiday
0,2019-10-01,4377,3210,0.270491,192.90625,4.826875,120,149,670,33,...,109,105,5,1868,Tuesday,0,1,0,0,0
1,2019-10-02,4272,3173,0.245378,354.53125,7.964375,110,158,672,39,...,103,107,3,1800,Wednesday,0,1,0,0,0
2,2019-10-03,4732,3722,0.249891,279.56250,5.829688,81,132,729,43,...,639,95,6,1899,Thursday,0,1,0,0,0
3,2019-10-04,3547,2619,0.236200,259.28125,5.872188,89,104,547,34,...,139,97,3,1438,Friday,0,1,0,0,0
4,2019-10-05,2073,1595,0.222594,246.18750,4.771875,60,64,315,16,...,95,58,4,741,Saturday,1,1,0,0,0


In [9]:
# we add time effects that indicate weekends, month, and holidays
time_effects = ['weekend', 'holiday'] + [col for col in df_final.columns if col.startswith('month')]

# multiple regression with number of cart users as the response variable
X = df_final[['user_new', 'user_tabletdesktop', 'user_other', 'user_affiliate',
              'user_direct', 'user_display', 'user_organic', 'user_paid', 'user_referral',
              'user_social', 'session_depth', 'session_duration'] + time_effects]
# response variable
y = df_final['user_addtocart']
# add constant so intercept is calculated too
X = sm.add_constant(X)
# run regression
lr = sm.OLS(y, X)
results = lr.fit()
# output results
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         user_addtocart   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.848
Method:                 Least Squares   F-statistic:                     31.27
Date:                Fri, 12 Nov 2021   Prob (F-statistic):           3.49e-27
Time:                        12:22:18   Log-Likelihood:                -357.37
No. Observations:                  93   AIC:                             750.7
Df Residuals:                      75   BIC:                             796.3
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -6.7182     23

(b)

In [10]:
# we want to run a regression with the probability of cart use as the response variable
# probability of cart use = number of users that add to cart / total number of users
df_final['p_addtocart'] = df_final['user_addtocart'] / df_final['user']

# run the same regression as above with this new response variable
X = df_final[['user_new', 'user_tabletdesktop', 'user_other', 'user_affiliate',
              'user_direct', 'user_display', 'user_organic', 'user_paid', 'user_referral',
              'user_social', 'session_depth', 'session_duration'] + time_effects]
y = df_final['p_addtocart']
X = sm.add_constant(X)
lr = sm.OLS(y, X)
results = lr.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            p_addtocart   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     4.959
Date:                Fri, 12 Nov 2021   Prob (F-statistic):           5.52e-07
Time:                        12:22:18   Log-Likelihood:                 406.51
No. Observations:                  93   AIC:                            -777.0
Df Residuals:                      75   BIC:                            -731.4
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0227      0

(c)

In [11]:
# odds ratio = probability / (1-probability) --> take log of this
df_final['lor_p_addtocart'] = np.log(df_final['p_addtocart'] / (1 - df_final['p_addtocart']))

# run the same regression as above with this new response variable
X = df_final[['user_new', 'user_tabletdesktop', 'user_other', 'user_affiliate',
              'user_direct', 'user_display', 'user_organic', 'user_paid', 'user_referral',
              'user_social', 'session_depth', 'session_duration'] + time_effects]
y = df_final['lor_p_addtocart']
X = sm.add_constant(X)
lr = sm.OLS(y, X)
results = lr.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:        lor_p_addtocart   R-squared:                       0.539
Model:                            OLS   Adj. R-squared:                  0.435
Method:                 Least Squares   F-statistic:                     5.160
Date:                Fri, 12 Nov 2021   Prob (F-statistic):           2.84e-07
Time:                        12:22:18   Log-Likelihood:                 57.058
No. Observations:                  93   AIC:                            -78.12
Df Residuals:                      75   BIC:                            -32.53
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -3.7482      0

(d)

In [12]:
# run binomial regression with the same predictors as above
# since this is a binomial regression, the response variable should be a probability between 0 and 1
# so we'll use p_addtocart

# run binomial regression
X = df_final[['user_new', 'user_tabletdesktop', 'user_other', 'user_affiliate',
              'user_direct', 'user_display', 'user_organic', 'user_paid', 'user_referral',
              'user_social', 'session_depth', 'session_duration'] + time_effects]
y = df_final['p_addtocart']
X = sm.add_constant(X)
br = sm.GLM(y, X, family=sm.families.Binomial(), link='logit')
results = br.fit()
print(results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:            p_addtocart   No. Observations:                   93
Model:                            GLM   Df Residuals:                       75
Model Family:                Binomial   Df Model:                           17
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -8.0992
Date:                Fri, 12 Nov 2021   Deviance:                     0.036328
Time:                        12:22:18   Pearson chi2:                   0.0369
No. Iterations:                     7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -3.7553     12

A regression of the log of the odds ratio on the predictor variables is the correct approach since the range of the response variable is from $-\infty$ to $\infty$. We'll thus apply this to the remaining response variables: purchase probability and purchase probability conditional on cart use.

(f)

In [13]:
# the log odds will go to -inf if there are no purchases, so we drop those days
df_final_nz = df_final[df_final['user_purchase'] != 0]
# probability of purchase = number of users that purchase / total number of users
df_final_nz['p_purchase'] = df_final_nz['user_purchase'] / df_final_nz['user']
# odds ratio = probability / (1-probability) --> take log of this
df_final_nz['lor_p_purchase'] = np.log(df_final_nz['p_purchase'] / (1 - df_final_nz['p_purchase']))

# run the same regression as above with this new response variable
X = df_final_nz[['user_new', 'user_tabletdesktop', 'user_other', 'user_affiliate',
              'user_direct', 'user_display', 'user_organic', 'user_paid', 'user_referral',
              'user_social', 'session_depth', 'session_duration'] + time_effects]
y = df_final_nz['lor_p_purchase']
X = sm.add_constant(X)
lr = sm.OLS(y, X)
results = lr.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:         lor_p_purchase   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.472
Method:                 Least Squares   F-statistic:                     5.776
Date:                Fri, 12 Nov 2021   Prob (F-statistic):           4.28e-08
Time:                        12:22:18   Log-Likelihood:                -39.129
No. Observations:                  92   AIC:                             114.3
Df Residuals:                      74   BIC:                             159.6
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -5.4232      0

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(g)

To analyze purchase probability conditional on cart use, we need to create a new purcahse probability column that--rather than being formed by dividing the number of users that purchase by all users--is formed by dividing the number of users that purchase by the number of users that add to cart.

In [14]:
# computing purchase probability conditional on adding to cart
df_final_nz['p_purchase|atc'] = df_final_nz['user_purchase'] / df_final_nz['user_addtocart']
# odds ratio = probability / (1-probability) --> take log of this
df_final_nz['lor_p_purchase|atc'] = np.log(df_final_nz['p_purchase|atc'] / (1 - df_final_nz['p_purchase|atc']))

# run the same regression as above with this new response variable
X = df_final_nz[['user_new', 'user_tabletdesktop', 'user_other', 'user_affiliate',
              'user_direct', 'user_display', 'user_organic', 'user_paid', 'user_referral',
              'user_social', 'session_depth', 'session_duration'] + time_effects]
y = df_final_nz['lor_p_purchase|atc']
X = sm.add_constant(X)
lr = sm.OLS(y, X)
results = lr.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:     lor_p_purchase|atc   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     3.573
Date:                Fri, 12 Nov 2021   Prob (F-statistic):           7.13e-05
Time:                        12:22:18   Log-Likelihood:                -51.558
No. Observations:                  92   AIC:                             139.1
Df Residuals:                      74   BIC:                             184.5
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -1.6217      0

C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\HP\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
